# Lab 12: Plate flexure and the Fourier transform
---

## Submission

Please upload your completed notebook (renamed with your name) to Canvas as usual.

## Introduction

In this lab, you will compute the expected plate flexural response for loading as represented by a bathymetric profile. You will decompose the bathymetric profile into a Fourier series and compute the response for each periodic component of the load, then sum these responses to obtain the full profile. This lab will 1) familiarize you with Python's basic FFT functionality (using the [`numpy.fft`](https://numpy.org/doc/stable/reference/routines.fft.html) module) and 2) give you practice computing flexural responses for periodic loads.

Read [this overview of the Fourier series](https://en.wikipedia.org/wiki/Fourier_series#Definition) if you need to review the mathematics. We summarize the essentials below.

## Fourier series basics

Fourier series break a periodic function down into a sum of cosines and sines of varying amplitudes and frequencies. The "Fourier coefficients" control the weighting given to each individual cosine or sine. For a continuous function $s(x)$ periodic on the interval $[-\lambda/2, \lambda/2]$ (i.e., with wavelength $\lambda$), the Fourier coefficients are given by

\begin{align}
a_n &= \frac{2}{\lambda}\int_{-\lambda/2}^{\lambda/2} s(x)\cos\left(\frac{2\pi n x}{\lambda}\right)\,\mathrm{d}x\\
b_n &= \frac{2}{\lambda}\int_{-\lambda/2}^{\lambda/2} s(x)\sin\left(\frac{2\pi n x}{\lambda}\right)\,\mathrm{d}x\,.
\end{align}

The function can be rebuilt via the infinite sum

$$s(x) = \frac{a_0}{2} + \sum_{n=1}^\infty \left[a_n \cos\left(\frac{2\pi nx}{\lambda}\right) + b_n \sin\left(\frac{2\pi nx}{\lambda} \right) \right].$$

Note that $b_0 = 0$ since $\sin(0) = 0$. Also, note that

$$
a_0 = \frac{2}{\lambda}\int_{-\lambda/2}^{\lambda/2} s(x)\,\mathrm{d}x\,,
$$

so

$$
\frac{a_0}{2} = \frac{1}{\lambda}\int_{-\lambda/2}^{\lambda/2} s(x)\,\mathrm{d}x
$$

is simply the average value of $s(x)$ on the interval $[-\lambda/2, \lambda/2]$. In this lab, we'll be using the fast Fourier transform to obtain the coefficients $a_n$ and $b_n$. This allows us to work with discrete signals $s_m = [s_1, s_2, \dots, s_N]$ instead of continuous signals $s(x)$. 

## 1. Examples of Fourier series analysis and synthesis using `numpy.fft.rfft()`

To begin this lab, please experiment with the following code cell, which introduces the process of determining the $a_n$ and $b_n$ coefficients (the **analysis** process) and the process of rebuilding the function from the $a_n$ and $b_n$ coefficients (the **synthesis** process). Change the value of `WAVE_TYPE` to specify different functions $s(x)$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import square


# Select 'sine' or 'square' for s(x)
WAVE_TYPE = 'sine'


if WAVE_TYPE == 'sine':
    N = 2 ** 3  # Signal length
    s = lambda x: np.sin(2 * np.pi * x)  # Sine wave with period = 1
elif WAVE_TYPE == 'square':
    N = 2 ** 6  # Signal length
    s = lambda x: square(2 * np.pi * x) + 2  # Square wave with period = 1, DC offset of 2
else:
    raise ValueError('Unrecognized input!')

x = np.arange(N) / N  # Define time vector from 0 (inclusive) to 1 (exclusive), our interval

sx = s(x)  # Compute function values

# Plot original function
fig, ax = plt.subplots()
ax.plot(x, sx, color='black', label='Original')

# KEY COMMAND: Take the fast Fourier transform
sx_fft = np.fft.rfft(sx)

# The Fourier series coefficients
a_n = (2 / N) * sx_fft.real
b_n = -(2 / N) * sx_fft.imag

# Now we can use the coefficients to build back our signal
sx_rc = np.empty((sx_fft.size, N))
for n in range(sx_fft.size):
    if n == 0:
        sx_rc[n, :] = (a_n[n] / 2) * np.ones(x.size)  # a_0 / 2
    else:
        sx_rc[n, :] = a_n[n] * np.cos(2 * np.pi * n * x) + b_n[n] * np.sin(2 * np.pi * n * x)

# Print the average value on the interval, a_0 / 2
print(f'a_0 / 2 = {a_n[0] / 2:.1f}')
        
# Compare our results
ax.plot(x, sx_rc.sum(axis=0), marker='x', color='red', linestyle='none', zorder=-1, label='Reconstructed')
ax.set_title(f'FFT of a {WAVE_TYPE} wave')
ax.set_xlabel('$x$')
ax.set_ylabel('$s(x)$')
ax.legend()
ax.set_xlim(0, 1)
plt.show()

## 2. Fourier transform of a bathymetric profile

Compute the Fourier transform coefficients $a_n$ and $b_n$ for the bathymetric profile given in the file `bathymetry.mat`. Plot the original bathymetry as well as the reconstructed profile, in addition to computing the coefficients.

The original bathymetric profile comes from [Watts et al. (2006)](https://agupubs.onlinelibrary.wiley.com/doi/epdf/10.1029/2005JB004083) but has been preprocessed for your use here. To load the data, run the following code cell, which loads in the $x$- and $y$-coordinates of the bathymetry in meters:

In [ ]:
from scipy.io import loadmat

!curl -s -O https://raw.githubusercontent.com/uafgeoteach/GEOS631_FoG/master/labs/lab_12/bathymetry.mat

data = loadmat('bathymetry.mat', squeeze_me=True)

xbathy = data['xbathy'] * 1000  # [m]
ybathy = data['ybathy']  # [m]

## 3. Computing the flexural response to arbitrary topography

Equations 3.110 and 3.111 in Turcotte and Schubert give the response to topography that varies according to $\sin(x)$. The equations for $\cos(x)$ are directly analogous (replace $\sin$ with $\cos$). Write a function or functions that compute these responses.

Now take the Fourier coefficients from part 2 and compute the flexural response using reasonable density estimates. Compute the response for the values of $T_\mathrm{e}$ used in Watts et al. (2006) (10, 25, and 50 km; see Figure 7b) and plot the deflection.

Explain the meaning of the parameter $T_\mathrm{e}$ in the context of your plots for $T_\mathrm{e} = $ 10 versus 25 versus 50 km.